In [20]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
import gc
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2018_Q3/YoY'

In [21]:
store_info=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20181201-135231-415.txt",sep="|",dtype=str)
store_info=store_info[['location_id','city_nm','state_nm','zip_cd','open_dt']]

store_info_2=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20181101-134628-331.txt",sep="|",dtype=str)
store_info_2=store_info_2[['location_id','city_nm','state_nm','zip_cd','open_dt']]
store_info_2=store_info_2[~store_info_2['location_id'].isin(store_info['location_id'].tolist())]
store_info=store_info.append(store_info_2)

store_info_2=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20181001-135417-132.txt",sep="|",dtype=str)
store_info_2=store_info_2[['location_id','city_nm','state_nm','zip_cd','open_dt']]
store_info_2=store_info_2[~store_info_2['location_id'].isin(store_info['location_id'].tolist())]
store_info=store_info.append(store_info_2)

store_info_2=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20180901-133640-935.txt",sep="|",dtype=str)
store_info_2=store_info_2[['location_id','city_nm','state_nm','zip_cd','open_dt']]
store_info_2=store_info_2[~store_info_2['location_id'].isin(store_info['location_id'].tolist())]
store_info=store_info.append(store_info_2)

store_info_2=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20180801-133641-576.txt",sep="|",dtype=str)
store_info_2=store_info_2[['location_id','city_nm','state_nm','zip_cd','open_dt']]
store_info_2=store_info_2[~store_info_2['location_id'].isin(store_info['location_id'].tolist())]
store_info=store_info.append(store_info_2)

store_info_2=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20180703.txt",sep="|",dtype=str)
store_info_2=store_info_2[['location_id','city_nm','state_nm','zip_cd','open_dt']]
store_info_2=store_info_2[~store_info_2['location_id'].isin(store_info['location_id'].tolist())]
store_info=store_info.append(store_info_2)

store_info_2=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20171115.txt",sep="|",dtype=str)
store_info_2=store_info_2[['location_id','city_nm','state_nm','zip_cd','open_dt']]
store_info_2=store_info_2[~store_info_2['location_id'].isin(store_info['location_id'].tolist())]
store_info=store_info.append(store_info_2)

store_info_2=pd.read_table("/home/jian/BigLots/static_files/Store_list/MediaStormStores20170913.txt",sep="|",dtype=str)
store_info_2=store_info_2[['location_id','city_nm','state_nm','zip_cd','open_dt']]
store_info_2=store_info_2[~store_info_2['location_id'].isin(store_info['location_id'].tolist())]
store_info=store_info.append(store_info_2)
store_info['zip_cd']=store_info['zip_cd'].apply(lambda x: x.split("-")[0].zfill(5))


DMA_Zip=pd.read_excel("/home/jian/Docs/Geo_mapping/Zips by DMA by County16-17 nielsen.xlsx",skiprows=1,dtype=str)
DMA_Zip=DMA_Zip.iloc[:,[0,2]]
DMA_Zip.columns=['zip_cd','DMA']
DMA_Zip=DMA_Zip.drop_duplicates(['zip_cd'])


store_info=pd.merge(store_info,DMA_Zip,on="zip_cd",how="left")

In [22]:
stores_sales_data=pd.read_excel("/home/jian/BiglotsCode/outputs/Output_2018-12-01/wide_sales_date2018-12-01.xlsx",dtype=str,sheetname="sales")
stores_trans_data=pd.read_excel("/home/jian/BiglotsCode/outputs/Output_2018-12-01/wide_sales_date2018-12-01.xlsx",dtype=str,sheetname="transactions")

Q3_Start_week_2018=datetime.date(2018,8,11)
Q3_End_week_2018=datetime.date(2018,11,3)
Q3_Start_week_2017=datetime.date(2017,8,12)
Q3_End_week_2017=datetime.date(2017,11,4)
Q3_2017_Weeks=[str(Q3_Start_week_2017+datetime.timedelta(days=7*i)) for i in range(13)]
Q3_2018_Weeks=[str(Q3_Start_week_2018+datetime.timedelta(days=7*i)) for i in range(13)]


stores_sales_data=stores_sales_data[['location_id']+Q3_2017_Weeks+Q3_2018_Weeks]
stores_trans_data=stores_trans_data[['location_id']+Q3_2017_Weeks+Q3_2018_Weeks]


for col in Q3_2017_Weeks+Q3_2018_Weeks:
    stores_sales_data[col]=stores_sales_data[col].astype(float)
    stores_trans_data[col]=stores_trans_data[col].astype(int)
    

stores_trans_data['2017_Q3_Trans']=stores_trans_data[Q3_2017_Weeks].sum(axis=1)
stores_trans_data['2018_Q3_Trans']=stores_trans_data[Q3_2018_Weeks].sum(axis=1)
stores_sales_data['2017_Q3_Sales']=stores_sales_data[Q3_2017_Weeks].sum(axis=1)
stores_sales_data['2018_Q3_Sales']=stores_sales_data[Q3_2018_Weeks].sum(axis=1)

for col in Q3_2017_Weeks+Q3_2018_Weeks:
    stores_sales_data=stores_sales_data.rename(columns={col:"Sales_"+col})
    stores_trans_data=stores_trans_data.rename(columns={col:"Trans_"+col})
    

df_1=pd.merge(store_info,stores_sales_data,on="location_id",how="left")
df_1=pd.merge(df_1,stores_trans_data,on="location_id",how="left")
del store_info
df_1['Q3_Sales_YoY']=(df_1['2018_Q3_Sales']-df_1['2017_Q3_Sales'])/df_1['2017_Q3_Sales']
df_1['Q3_Trans_YoY']=(df_1['2018_Q3_Trans']-df_1['2017_Q3_Trans'])/df_1['2017_Q3_Trans']
del store_info_2
gc.collect()
df_1.shape

(1472, 64)

In [24]:
df_1['location_id'][0]

'1'

In [6]:
rewards_sales_from_SP=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/From_Sp/combinedtransactions_0811.csv",
                                 dtype=str,usecols=['customer_id_hashed','transaction_date','transaction_id','location_id','total_transaction_amt','merch_cat'])
rewards_removed=rewards_sales_from_SP[rewards_sales_from_SP['merch_cat']=="80040410"]
rewards_sales_from_SP=rewards_sales_from_SP[rewards_sales_from_SP['merch_cat']!="80040410"]
del rewards_sales_from_SP['merch_cat']

rewards_sales_from_SP=rewards_sales_from_SP.drop_duplicates()

rewards_sales_from_SP['transaction_date']=rewards_sales_from_SP['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())

rewards_sales_from_SP_2017Q3=rewards_sales_from_SP[(rewards_sales_from_SP['transaction_date']>=(Q3_Start_week_2017-datetime.timedelta(days=6))) & (rewards_sales_from_SP['transaction_date']<=Q3_End_week_2017)]
del rewards_sales_from_SP
gc.collect()


rewards_sales_from_SP_2017Q3['total_transaction_amt']=rewards_sales_from_SP_2017Q3['total_transaction_amt'].astype(float)
rewards_sales_from_SP_2017Q3=rewards_sales_from_SP_2017Q3.drop_duplicates()
gc.collect()
rewards_sales_from_SP_2017Q3.shape

(13727931, 5)

In [7]:
file_list=['/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20180814-131127-683.txt',
          '/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20180828-131144-584.txt',
          '/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20180911-133008-590.txt',
          '/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20180925-132958-543.txt',
          '/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20181009-132856-945.txt',
          '/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20181023-132316-829.txt',
          '/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20181106-132358-296.txt']

In [8]:
biweek_rewards_2018=pd.DataFrame()
removed_2018=pd.DataFrame()
for file_i in file_list:
    df=pd.read_table(file_i,dtype=str,sep="|")
    df_removed=df[df['merch_cat']=="80040410"]
    removed_2018=removed_2018.append(df_removed)
    df=df[df['merch_cat']!="80040410"]
    del df['merch_cat']
    del df['total_transaction_units']
    df=df.drop_duplicates()
    df['transaction_dt']=df['transaction_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
    biweek_rewards_2018=biweek_rewards_2018.append(df)
    print(datetime.datetime.now())
biweek_rewards_2018.shape

2018-12-04 17:20:58.574303
2018-12-04 17:21:43.440107
2018-12-04 17:22:29.292508
2018-12-04 17:23:10.643475
2018-12-04 17:24:02.211847
2018-12-04 17:24:44.561800
2018-12-04 17:25:30.436441


(18834515, 6)

In [9]:
print("check_point_2")

check_point_2


In [12]:
biweek_rewards_2018['total_transaction_amt']=biweek_rewards_2018['total_transaction_amt'].astype(float)
historical_rewards_2017=rewards_sales_from_SP_2017Q3.rename(columns={"transaction_date":"transaction_dt"})
del rewards_sales_from_SP_2017Q3

rewards_by_day_2017_sales=historical_rewards_2017.groupby(['location_id','transaction_dt'])['total_transaction_amt'].sum().to_frame().reset_index()
rewards_by_day_2017_sales=rewards_by_day_2017_sales.rename(columns={"total_transaction_amt":"rewards_sales"})
rewards_by_day_2018_sales=biweek_rewards_2018.groupby(['location_id','transaction_dt'])['total_transaction_amt'].sum().to_frame().reset_index()
rewards_by_day_2018_sales=rewards_by_day_2018_sales.rename(columns={"total_transaction_amt":"rewards_sales"})

rewards_by_day_2017_trans=historical_rewards_2017.groupby(['location_id','transaction_dt'])['total_transaction_amt'].count().to_frame().reset_index()
rewards_by_day_2017_trans=rewards_by_day_2017_trans.rename(columns={"total_transaction_amt":"rewards_trans"})
rewards_by_day_2018_trans=biweek_rewards_2018.groupby(['location_id','transaction_dt'])['total_transaction_amt'].count().to_frame().reset_index()
rewards_by_day_2018_trans=rewards_by_day_2018_trans.rename(columns={"total_transaction_amt":"rewards_trans"})



'1'

In [13]:
def week_end_dt(x):
    if x.weekday()==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=(5-x.weekday()))
    return y
rewards_by_day_2017_sales['week_end_dt']=rewards_by_day_2017_sales['transaction_dt'].apply(lambda x: week_end_dt(x))
rewards_by_day_2018_sales['week_end_dt']=rewards_by_day_2018_sales['transaction_dt'].apply(lambda x: week_end_dt(x))

rewards_by_day_2017_trans['week_end_dt']=rewards_by_day_2017_trans['transaction_dt'].apply(lambda x: week_end_dt(x))
rewards_by_day_2018_trans['week_end_dt']=rewards_by_day_2018_trans['transaction_dt'].apply(lambda x: week_end_dt(x))

In [15]:
rewards_by_week_2017_sales=rewards_by_day_2017_sales.groupby(['location_id','week_end_dt'])['rewards_sales'].sum().to_frame().reset_index()
rewards_by_week_2018_sales=rewards_by_day_2018_sales.groupby(['location_id','week_end_dt'])['rewards_sales'].sum().to_frame().reset_index()

rewards_by_week_2017_trans=rewards_by_day_2017_trans.groupby(['location_id','week_end_dt'])['rewards_trans'].sum().to_frame().reset_index()
rewards_by_week_2018_trans=rewards_by_day_2018_trans.groupby(['location_id','week_end_dt'])['rewards_trans'].sum().to_frame().reset_index()

rewards_by_week_both_sales=rewards_by_week_2017_sales.append(rewards_by_week_2018_sales)
rewards_by_week_both_trans=rewards_by_week_2017_trans.append(rewards_by_week_2018_trans)

In [32]:
rewards_by_week_both_sales_wide=rewards_by_week_both_sales.pivot_table(index="location_id",columns="week_end_dt",values="rewards_sales").reset_index()
rewards_by_week_both_trans_wide=rewards_by_week_both_trans.pivot_table(index="location_id",columns="week_end_dt",values="rewards_trans").reset_index()

rewards_by_week_both_sales_wide.columns=[str(x) for x in rewards_by_week_both_sales_wide.columns.tolist()]
rewards_by_week_both_trans_wide.columns=[str(x) for x in rewards_by_week_both_trans_wide.columns.tolist()]

In [33]:
for col in Q3_2017_Weeks+Q3_2018_Weeks:
    rewards_by_week_both_sales_wide=rewards_by_week_both_sales_wide.rename(columns={col:"R_Sales_"+col})
    rewards_by_week_both_trans_wide=rewards_by_week_both_trans_wide.rename(columns={col:"R_Trans_"+col})
    
df_1=pd.merge(df_1,rewards_by_week_both_sales_wide,on="location_id",how="left")
df_1=pd.merge(df_1,rewards_by_week_both_trans_wide,on="location_id",how="left")
df_1=df_1.fillna(0)

In [37]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2018_Q3/YoY/BL_rewards_by_store_week_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
df_1.to_excel(writer,"data",index=False)
rewards_removed.to_excel(writer,"exclusion_rewards_2017",index=False)
removed_2018.to_excel(writer,"exclusion_rewards_2018",index=False)
writer.save()



In [34]:
df_1.shape

(1472, 118)